# Using NIDN with a finite-difference time-domain (FDTD) solver 

This notebook gives explanatory examples on how to use NIDN with FDTD simulations. For more on how FDTD simulations work, see Running_FDTD notebook or the FDTD section in the NIDN docs. 

We start by importing the nidn, or if nidn is not installed append the root module folder to get access to nidn 

In [ ]:
# Append root folder in case you haven't installed NIDN
import sys
sys.path.append("../")

import nidn
import torch
nidn.set_log_level("INFO")

# Inverse Design of a uniform Titanium Dioxide Layer

Here, we define the target spectrum as a spectrum from a uniform TiO2 layer which we compute, and test if NIDN is able to reconstruct the permittivity of the TiO2 layer.

In [ ]:
# Load default cfg as starting point

cfg = nidn.load_default_cfg()

# Specify grid setup
cfg.Nx = 1 # Set layer size  to 1x1 (interpreted as uniform)
cfg.Ny = 1
cfg.N_layers = 1 # Choose number of layers

# Specify your desired range of wavelengths
cfg.physical_wavelength_range[0] = 3e-7
cfg.physical_wavelength_range[1] = 9e-7
cfg.PER_LAYER_THICKNESS = [0.38]

# Let's investigate 16 frequency points
cfg.N_freq = 16

# use FDTD
cfg.solver = "FDTD"

# Some FDTD specific settings, see docs for details
cfg.FDTD_min_gridpoints_per_unit_magnitude = 32
cfg.FDTD_niter = int(800 *  cfg.FDTD_min_gridpoints_per_unit_magnitude / 50)
cfg.FDTD_pulse_type = 'continuous'
cfg.FDTD_source_type = 'line'

cfg.target_frequencies = nidn.compute_target_frequencies(
    cfg.physical_wavelength_range[0],
    cfg.physical_wavelength_range[1],
    cfg.N_freq,
    cfg.freq_distribution
)

# We Compute ground truth
eps_grid = torch.zeros(cfg.Nx,cfg.Ny,cfg.N_layers,cfg.N_freq,dtype=torch.cfloat)
layer_builder = nidn.LayerBuilder(cfg)
eps_grid[:,:,0,:] = layer_builder.build_uniform_layer("titanium_oxide")
# Compute spectrum for this configuration
R,T = nidn.compute_spectrum(eps_grid,cfg)
nidn.plot_spectrum(cfg,R,T,show_absorption=True)

cfg.target_reflectance_spectrum = R
cfg.target_transmittance_spectrum = T

physical_wls, normalized_freqs = nidn.get_frequency_points(cfg)
print("Physical wavelengths are (in meters):")
print(physical_wls)

## Example 1 - Uniform single-layer with restricted epsilon

Let's start with a uniform single-layer and see if NIDN can get sufficiently close to the ground truth.

In [ ]:
cfg.pop("model",None); # Forget any old model

# Allowed range of epsilon values
cfg.real_min_eps = 0.0
cfg.real_max_eps = 20.0
cfg.imag_min_eps = 0.0
cfg.imag_max_eps = 6.0

# Choose model type, regression or classification
cfg.type = "regression" 

In [ ]:
#Show all used settings
nidn.print_cfg(cfg)

`print_cfg(cfg)` shows you more or less everything you want to know about the config.
Using `run_training(cfg)`, we run the network until it reaches the number of iterations set above (or until you interrupt it).

In [ ]:
# Set number of training iterations (that is forward model evaluations) to perform
# Note that this will be pretty slow for now, 200 iterations lead to a good result
# but that requires hours
cfg.iterations = 10  
nidn.run_training(cfg);

### Interpretation of results

#### Loss plot

The loss as a function of model evaluations is presented below. As the training evolves, the three losses here, [L1](https://afteracademy.com/blog/what-are-l1-and-l2-loss-functions), Loss, and Weighted Average Loss, can be seen to decrease. For this regression case, the L1 and Loss are the same.

In [ ]:
nidn.plot_losses(cfg)

#### Spectrum plots

The produced RTA spectra are plotted together with the target spectra in the figure below.

In [ ]:
nidn.plot_spectra(cfg)

#### Absolute grid values plot

The complex absolute value of the epsilon over all frequencies is presented here. This plot is in general more useful for patterned multilayers.

In [ ]:
nidn.plot_model_grid(cfg)

#### Epsilon vs frequency and real materials

The following function plots the epsilon values vs. frequency of grid points against real materials in our library. This plot is in general more useful for patterned multilayers.

In [ ]:
nidn.plot_eps_per_point(cfg)

## Example 2 - Uniform single-layer with materials classification

Next up is the same example, a uniform single-layer of titanium oxide, but this time we check if NIDN can predict the correct material.

In [ ]:
cfg.pop("model",None); # Forget the old model
cfg.Nx = 1 # Set layer size  to 1x1 (interpreted as uniform)
cfg.Ny = 1
cfg.N_layers = 1 # Choose number of layers

cfg.type = "classification" # Choose type as described above
cfg.iterations = 20 # Set number of training iterations (that is forward model evaluations) to perform

In [ ]:
nidn.run_training(cfg);

In [ ]:
nidn.plot_losses(cfg)
nidn.plot_spectra(cfg)
nidn.plot_model_grid(cfg)
nidn.plot_eps_per_point(cfg)